In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tvm
from tvm import relay
from tvm.relay import testing
import tvm.auto_scheduler as auto_scheduler
from tvm import autotvm

# Robot Vision architecture
class Net(nn.Module):
    def __init__(self,
                 conv_layers=2,
                 linear_layers=3,
                 in_channels=4,
                 out_channels=3,
                 kernel_size=5,
                 h_dim=120,
                 stride=1,
                 padding=0,
                 dilation=1):
        super(Net, self).__init__()
        self.in_channels=in_channels
        self.out_channels=out_channels
        self.kernel_size=kernel_size
        self.stride=stride
        self.padding=padding
        self.dilation=dilation
        self.h_dim=h_dim
        self.convs = torch.nn.Sequential()
        self.linear = torch.nn.Sequential()
        self.conv_layers=conv_layers
        self.linear_layers=linear_layers

        in_c = in_channels
        out_c = out_channels
        for layer in range(conv_layers):
            self.convs.add_module("convs%d" % (layer),nn.Conv2d(in_c, out_c,kernel_size))
            self.convs.add_module("convs_relu%d" % (layer), nn.ReLU())
            self.convs.add_module("pool%d" % layer, nn.MaxPool2d(2,2))
            in_c = out_c

        in_d = 8427
        for layer in range(linear_layers-1):
            self.linear.add_module("mlp_%d" % (layer),nn.Linear(in_d,h_dim))
            self.linear.add_module("mlp_relu%d" % (layer), nn.ReLU())
            in_d = h_dim
        self.linear.add_module("mlp_%d" % (layer + 1), nn.Linear(in_d, 12))

    def forward(self,x):
        x = self.convs(x)
        x = torch.flatten(x, 1)
        x = self.linear(x)
        return x

In [2]:
model = Net()
model.load_state_dict(torch.load('robotVis.pth', weights_only=True) ) # Import robot vision model
model = model.eval()

input_dimension = [1, 4, 224, 224] 
input_data = torch.randn(input_dimension)
script = torch.jit.trace(model, input_data)
mod, params = relay.frontend.from_pytorch(script, [("input", input_dimension)])

In [3]:
target = tvm.target.Target("llvm")

tasks, task_weights = auto_scheduler.extract_tasks(mod, params, target)

In [4]:
tuner = auto_scheduler.TaskScheduler(tasks, task_weights)

tuner.tune(auto_scheduler.TuningOptions(num_measure_trials=7,measure_callbacks=[auto_scheduler.RecordToFile("tuning_log.json")]))

|  ID  |                       Task Description                        | Latency (ms) | Speed (GFLOPS) | Trials |----------------------------------------------------------------------
------------------------------  [ Task Scheduler ]
----------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------
|    0 |                          vm_mod_fused_nn_conv2d_add_nn_relu_1 |            - |              - |      0 |
|    1 |                                     vm_mod_fused_nn_dense_add |            - |              - |      0 |
|    2 |                                  vm_mod_fused_nn_max_pool2d_1 |            - |              - |      0 |
|    3 |                            vm_mod_fused_nn_conv2d_add_nn_relu |            - |              - |      0 |
|    4 |                             vm_mod_fused_nn_dense_add_nn_relu |            - |              - |      0 |
|    5 | 

In [7]:
with auto_scheduler.ApplyHistoryBest("tuning_log.json"):
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
